# Youtube Data Collection and Processing

### Collection with googleapiclient and requests libraries.

First - build a dataframe with vidio ID, title, short description and category.

In [ ]:
### Imports
from googleapiclient.discovery import build
import requests
import pandas as pd


api_key = "*******************************"
youtube_api = build('youtube','v3', developerKey = api_key)

In [ ]:
### Initiate empty lists
titles = []
short_descriptions = []
ids = []
category = []
## q- is the query you wand to search in youtube
req = youtube_api.search().list(
                            q='technical ted talk',
                            order="relevance", 
                            part='snippet', 
                            type='videos', 
                            maxResults = 50)
res = req.execute()
while(len(titles)<no_of_samples):
    for i in range(len(res['items'])):
        titles.append(res['items'][i]['snippet']['title'])
        short_descriptions.append(
              res['items'][i]['snippet']['description'])
        try: 
            ids.append(res['items'][i]['id']['videoId'])
        except:
            ids.append('')
        category.append('Ted_talk')
            
    if('nextPageToken' in res):
        next_page_token = res['nextPageToken']
        req = youtube_api.search().list(
                                   q='flirting ted talk',
                                   order="relevance",
                                   part='snippet', 
                                   type='videos', 
                                   maxResults = 50, 
                                   pageToken=next_page_token)
        res = req.execute()
    else:
        break
### Construct Dataset with the lists
final_titles = titles
final_descriptions = short_descriptions
final_ids = ids
### Build the dataframe
data = pd.DataFrame({'Video Id': final_ids, 'Title': final_titles, 'Description': final_descriptions, 'Category': category})

Second - for each Video ID extract the full description

In [ ]:
### import
import requests

In [ ]:
full_description = []
for idx in data['Video Id']:
    r = requests.get(f'https://www.googleapis.com/youtube/v3/videos?  part=snippet&id={idx}&key={api_key}')
    des = r.json()['items'][0]['snippet']['description']
    full_description.append(des)
data['full_description'] = full_description

## Cleaning the data

In [ ]:
def clean_brackets(pred):
    """
    This function removes brackets and the contant inside.
    Input: pred - any text, here a summary.
    Output: cleaned_pred - without brackets and contant inside. 
    """
    round_brackets = re.findall(r"\(([A-Za-z0-9_]+)\)", pred)
    square_brackets = re.findall(r"\[([A-Za-z0-9_]+)\]", pred)
    cleaned_pred = pred
    if round_brackets:
        for match in round_brackets:
            word_match = "(" + match + ")"
            cleaned_pred = cleaned_pred.replace(word_match,"")
    if square_brackets:
        for match in round_brackets:
            word_match = "(" + match + ")"
            cleaned_pred = cleaned_pred.replace(word_match,"")
    return cleaned_pred

In [ ]:
def delete_words(input):
    """
    This function removes specific words.
    Input: a string.
    Output: clean_input - the string without the words. 
    """
    delete_word = ["click","|"]
    word_list = input.split(" ")
    clean_words = []
    for orig_word in word_list:
        if not orig_word in delete_word:
            clean_words.append(orig_word)
    clean_input = " ".join(clean_words)
    return clean_input

In [ ]:
def delete_http(input):
    """
    This function removes http.
    Input: a string.
    Output: clean_input - the string without the url. 
    """
    word_list = input.split(" ")
    clean_words = []
    for orig_word in word_list:
        if ("http" in orig_word) == False:
            clean_words.append(orig_word)
    clean_input = " ".join(clean_words)
    return clean_input

In [ ]:
def clean_hashtags(input):
    """
    This function removes hastags.
    Input: a string.
    Output: clean_input - the string without the hastag. 
    """
    word_list = input.split(" ")
    clean_words = []
    for word in word_list:
        if word.startswith("#") == False:
            clean_words.append(word)
    clean_input = " ".join(clean_words)
    return clean_input

In [ ]:
def clean_non_english(input):
    """
    This function removes non-English words as well as emojis.
    Input: a string.
    Output: clean_input - the string without the non-English words. 
    """
    word_list = input.split(" ")
    clean_words = []
    for word in word_list:
        if word.isascii() == True:
            clean_words.append(word)
    clean_input = " ".join(clean_words)
    return clean_input

In [ ]:
### Note - make sure to insert your relevat words in "delete_sent" to delete sentances with these words. 

import re
def clean_data(input_txt):
    delete_sent = ["subscribe","watch more","facebook","instagram","twitter","tumblr","newsletter","website","merch","youtube","google","ebook",
             "git-hub","tutorial","games","gameplay","coupon","news","read more",
             "follow", "abc","cbs","msnbc","business insider","bnc","cbc","fox","kcra"]
    delete_word = ["click here"]
    list_input_txt = sent_tokenize(input_txt)
    clean_list = []
    for sen in list_input_txt:
        if any(word.lower() in sen.lower() for word in delete_sent) == False:
            clean_list.append(sen)    
    cleaned_sentences = []
    for sen in clean_list:
        # else:
        """
        1. remove specific words
        2. remove non English words, emogi
        3. remove URLs
        4. remove #
        5. remove brackets
        6. remove objects that are not punctuation |
        """
        sen = clean_brackets(sen)
        sen = delete_words(sen)
        sen = delete_http(sen)
        sen = clean_hashtags(sen)
        sen = clean_non_english(sen)
        cleaned_sentences.append(sen)
    cleaned_input = " ".join(cleaned_sentences)
    return cleaned_input

In [ ]:
clean_data = data['Full Description'].apply(clean_data)